# LLMCheckerChain
This notebook showcases how to use LLMCheckerChain.

In [1]:
from langchain.chains import LLMCheckerChain, LLMChain, SequentialChain, ParallelChain
from langchain.llms import OpenAI
from langchain.prompts.base import ListOutputParser
from langchain.llms.base import LLM


llm = OpenAI(temperature=0.7)

text = "What type of mammal lays the biggest eggs?"

checker_chain = LLMCheckerChain(llm=llm, verbose=True)

# checker_chain.run(text)

In [2]:
# flake8: noqa
from langchain.prompts.prompt import PromptTemplate

_CREATE_DRAFT_ANSWER_TEMPLATE = """{question}\n\n"""
CREATE_DRAFT_ANSWER_PROMPT = PromptTemplate(
    input_variables=["question"], template=_CREATE_DRAFT_ANSWER_TEMPLATE
)

_LIST_ASSERTIONS_TEMPLATE = """Here is a statement:
{statement}
Make a bullet point list of the assumptions you made when producing the above statement.\n\n"""
LIST_ASSERTIONS_PROMPT = PromptTemplate(
    input_variables=["statement"], template=_LIST_ASSERTIONS_TEMPLATE
)

_CHECK_ASSERTIONS_TEMPLATE = """{assertions}
Repeat the assertion above, and then determine if it is true or false. If it is false, explain why.\n\n"""
CHECK_ASSERTIONS_PROMPT = PromptTemplate(
    input_variables=["assertions"], template=_CHECK_ASSERTIONS_TEMPLATE
)

_REVISED_ANSWER_TEMPLATE = """{checked_assertions}

Question: In light of the above assertions and checks, how would you answer the question '{question}'?

Answer:"""
REVISED_ANSWER_PROMPT = PromptTemplate(
    input_variables=["checked_assertions", "question"],
    template=_REVISED_ANSWER_TEMPLATE,
)

In [7]:
llm = OpenAI(temperature=0.7)

create_draft_answer_chain = ParallelChain(
    llm=llm, prompt=CREATE_DRAFT_ANSWER_PROMPT, output_key="statement"
)

list_assertions_chain = ParallelChain(
    llm=llm,
    prompt=LIST_ASSERTIONS_PROMPT,
    output_key="assertions",
    # This indicates that there are multiple outputs
    output_parser=lambda x: [i.strip() for i in x.split("\n") if len(i.strip()) > 0],
)

check_assertions_chain = ParallelChain(
    llm=llm, prompt=CHECK_ASSERTIONS_PROMPT, output_key="checked_assertions"
)

revised_answer_chain = ParallelChain(
    llm=llm,
    prompt=REVISED_ANSWER_PROMPT,
    output_key="revised_answer",
    # This indicates that there are multiple inputs
    input_parser=lambda x: "\n".join(x),
)

self_ask_verification_chain = SequentialChain(
    chains=[
        create_draft_answer_chain,
        list_assertions_chain,
        check_assertions_chain,
        revised_answer_chain,
    ],
    input_variables=["question"],
    output_variables=["revised_answer"],
    verbose=True,
)

In [8]:
%%time
self_ask_verification_chain.run(text)



> Entering new SequentialChain chain...
Chain 0:
{'statement': ['\nThe largest eggs laid by a mammal belong to the whale shark, which can lay eggs up to 19 inches (48 cm) long.']}

Chain 1:
{'assertions': ['• Whale sharks lay eggs', '• The eggs are the largest of any mammal', '• The eggs can be up to 19 inches (48 cm) long']}

Chain 2:
{'checked_assertions': ['The assertion is: Whale sharks lay eggs.\nThis is false. Whale sharks are viviparous, meaning they give birth to live young. They do not lay eggs.', 'The eggs of any mammal are the largest eggs. FALSE. Mammals do not lay eggs, only birds and reptiles do.', 'The eggs can be up to 19 inches (48 cm) long. TRUE']}

Chain 3:
{'revised_answer': [' None, mammals do not lay eggs.']}


> Finished SequentialChain chain.
CPU times: user 312 ms, sys: 20 ms, total: 332 ms
Wall time: 4.58 s


[' None, mammals do not lay eggs.']

In [27]:
llm = OpenAI(temperature=0.7)

create_draft_answer_chain = ParallelChain(
    llm=llm, prompt=CREATE_DRAFT_ANSWER_PROMPT, output_key="statement", n = 25
)

list_assertions_chain = ParallelChain(
    llm=llm,
    prompt=LIST_ASSERTIONS_PROMPT,
    output_key="assertions",
    # This indicates that there are multiple outputs
#     output_parser=lambda x: [i.strip() for i in x.split("\n") if len(i.strip()) > 0],
)

check_assertions_chain = ParallelChain(
    llm=llm, prompt=CHECK_ASSERTIONS_PROMPT, output_key="checked_assertions"
)

revised_answer_chain = ParallelChain(
    llm=llm,
    prompt=REVISED_ANSWER_PROMPT,
    output_key="revised_answer",
    # This indicates that there are multiple inputs
#     input_parser=lambda x: "\n".join(x),
)


MOE_CONSENSUS_PROMPT = PromptTemplate(
    input_variables=["revised_answer", "question"],
    template="""{revised_answer}

Question: A group of experts each gave their opinion above. How would the plurality of them answer the question '{question}'?

Answer:""",
)

moe_answer_chain = ParallelChain(
    llm=llm,
    prompt=MOE_CONSENSUS_PROMPT,
    output_key="final_answer",
    # This indicates that there are multiple inputs
    input_parser=lambda x: "\n -" + "\n -".join(x),
)


self_ask_verification_chain = SequentialChain(
    chains=[
        create_draft_answer_chain,
        list_assertions_chain,
        check_assertions_chain,
        revised_answer_chain,
        moe_answer_chain,
    ],
    input_variables=["question"],
    output_variables=["final_answer"],
    verbose=True,
)

In [ ]:
%%time
self_ask_verification_chain.run(text)



> Entering new SequentialChain chain...
Chain 0:
{'statement': ['\n\nThe largest eggs laid by any mammal belong to the Elephantidae family, which includes the African elephant and the Asian elephant.', '\n\nThe platypus is the only mammal that lays eggs, and they are the largest of any mammal eggs.', '\n\nThe largest eggs laid by any mammal belong to the elephant, which can weigh up to 4 pounds.', '\nThe largest mammal egg belongs to the elephant.', '\n\nThe largest mammal that lays eggs is the elephant bird, an extinct species of flightless bird that was native to Madagascar.', '\nThe largest mammal that lays eggs is the the echidna, also known as the spiny anteater.', '\n\nThe biggest eggs laid by a mammal belong to the elephant. They are around the size of a grapefruit and are not fertilized.', '\nThe largest egg laid by a mammal is the egg of the elephant.', '\n\nThe biggest eggs laid by a mammal are those of the Australian Echidna, a species of spiny anteater.', '\nThe largest m